### Matrix factorization under Pytorch
I follow the [tutorial](http://blog.ethanrosenthal.com/2017/06/20/matrix-factorization-in-pytorch/) to code matrix factorization under PyTorch 0.4.0 and Python 3.5

In [1]:
import numpy as np
from scipy.sparse import rand as sprand
import torch
from torch.autograd import Variable

n_users = 1000
n_items = 1000
ratings = sprand(n_users, n_items, 
                 density=0.01, format='csr')
ratings.data = (np.random.randint(1, 5, 
                                  size=ratings.nnz)
                          .astype(np.float64))
ratings = ratings.toarray()

In [4]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, 
                                               n_factors,
                                               sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, 
                                               n_factors,
                                               sparse=True)
        
    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

In [5]:
model = MatrixFactorization(n_users, n_items, n_factors=20)

In [6]:
# Loss function
loss_func = torch.nn.MSELoss()

In [7]:
# Define optimization technique
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=1e-6) # learning rate

In [8]:
# Sort our data
rows, cols = ratings.nonzero()
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]

for row, col in zip(*(rows, cols)):
    # Turn data into variables
    rating = Variable(torch.FloatTensor([ratings[row, col]]))
    row = Variable(torch.LongTensor([np.long(row)]))
    col = Variable(torch.LongTensor([np.long(col)]))
    
    # Predict and calculate loss
    prediction = model(row, col)
    loss = loss_func(prediction, rating)
    
    # Backpropagate
    loss.backward()
    
    # Update the parameters
    optimizer.step()

In [10]:
# print user latent factor
print(model.user_factors.weight)

Parameter containing:
tensor([[-0.6602,  0.2753,  0.7919,  ..., -0.6638, -0.4608, -1.0459],
        [ 0.1122,  0.9136,  1.4134,  ...,  0.2170,  0.4223, -1.6526],
        [-1.0616,  0.9610,  0.6030,  ..., -1.5356,  0.8040,  0.0736],
        ...,
        [ 0.3877, -0.6013, -0.0516,  ..., -0.3256,  0.0125, -0.9368],
        [ 0.3810, -2.1886,  1.0221,  ...,  0.2800,  0.6728,  0.8364],
        [-0.0005,  0.5182,  1.0412,  ..., -0.9755,  0.3677,  1.6703]],
       requires_grad=True)


In [11]:
# print venue latent factor
print(model.item_factors.weight)

Parameter containing:
tensor([[ 0.1698, -1.7126,  0.4861,  ...,  0.4945,  0.8796,  0.7173],
        [ 1.0134,  1.7968,  0.4016,  ..., -0.5096,  1.0018, -0.1825],
        [ 1.2538, -0.1432,  0.0577,  ..., -0.2004, -1.2428, -0.2907],
        ...,
        [-1.0364,  0.8970,  0.6789,  ..., -0.3816,  1.0716, -0.3105],
        [-0.5187, -0.2775,  0.9966,  ..., -1.4349, -1.0049,  0.0762],
        [ 0.8415, -2.3906, -0.2427,  ...,  1.8611,  0.6607, -0.4663]],
       requires_grad=True)
